In [261]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
import sys
import torch.nn.functional as F

np.random.seed(1)

device = 'cpu'

df = pd.read_csv('b_depressed.csv')

In [262]:
Y = df['depressed']
X = df.drop(columns=['depressed','no_lasting_investmen'], axis=1)

In [263]:
from sklearn.model_selection import train_test_split

X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.3, random_state=1)

In [264]:
X_train_tensor = torch.from_numpy(X_train.values).type(torch.FloatTensor)
Y_train_tensor = torch.from_numpy(Y_train.values).type(torch.FloatTensor)
X_valid_tensor = torch.from_numpy(X_valid.values).type(torch.FloatTensor)
Y_valid_tensor = torch.from_numpy(Y_valid.values).type(torch.FloatTensor)
#X_train_tensor = X_train_tensor.unsqueeze(dim=0)
Y_train_tensor = Y_train_tensor.unsqueeze(dim=1)
# Y_train_tensor

In [265]:
X_train_tensor.shape, Y_train_tensor.shape

(torch.Size([1000, 21]), torch.Size([1000, 1]))

In [266]:
class CustomModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(21,20),
            nn.Linear(20,8),
            nn.Linear(8,1)
        )
    def forward(self, x):
        x = self.layer(x)
        return x

In [281]:
model = CustomModel()
# optimizer 설정
optimizer = optim.Adam(model.parameters(), lr=0.00001)
loss_func = torch.nn.BCEWithLogitsLoss()
nb_epochs = 3500
losses = []
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = model(X_train_tensor)

    # cost 계산
    cost = loss_func(hypothesis, Y_train_tensor)
    losses.append(cost.item())
    # for row in model.parameters():
    #     print(row)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 50 == 0:
        prediction = hypothesis >= torch.FloatTensor([0.5]) # 예측값이 0.5를 넘으면 True로 간주
        correct_prediction = prediction.float() == Y_train_tensor # 실제값과 일치하는 경우만 True로 간주
        accuracy = correct_prediction.sum().item() / len(correct_prediction) # 정확도를 계산
        print('Epoch {:4d}/{} Cost: {:.6f} Accuracy {:2.2f}%'.format( # 각 에포크마다 정확도를 출력
            epoch, nb_epochs, cost.item(), accuracy * 100,
        ))

Epoch    0/3500 Cost: 618223.750000 Accuracy 65.40%
Epoch   50/3500 Cost: 576936.437500 Accuracy 66.70%
Epoch  100/3500 Cost: 543255.875000 Accuracy 68.00%
Epoch  150/3500 Cost: 516026.625000 Accuracy 69.00%
Epoch  200/3500 Cost: 492249.906250 Accuracy 69.80%
Epoch  250/3500 Cost: 469790.437500 Accuracy 70.20%
Epoch  300/3500 Cost: 447933.406250 Accuracy 70.50%
Epoch  350/3500 Cost: 426550.093750 Accuracy 70.60%
Epoch  400/3500 Cost: 405525.625000 Accuracy 70.80%
Epoch  450/3500 Cost: 385007.156250 Accuracy 71.10%
Epoch  500/3500 Cost: 365050.500000 Accuracy 71.30%
Epoch  550/3500 Cost: 345576.875000 Accuracy 71.50%
Epoch  600/3500 Cost: 326434.968750 Accuracy 71.60%
Epoch  650/3500 Cost: 307626.500000 Accuracy 71.80%
Epoch  700/3500 Cost: 289131.593750 Accuracy 72.00%
Epoch  750/3500 Cost: 270834.343750 Accuracy 72.10%
Epoch  800/3500 Cost: 252815.640625 Accuracy 72.00%
Epoch  850/3500 Cost: 235340.546875 Accuracy 72.00%
Epoch  900/3500 Cost: 218012.171875 Accuracy 72.00%
Epoch  950/3

In [282]:
model.eval()
outputs = None

with torch.no_grad():
    outputs = model(X_valid_tensor)
    loss = loss_func(outputs, torch.Tensor(Y_valid_tensor).reshape(-1, 1))

print("test loss :", loss)

test loss : tensor(337.0468)
